In [1]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

import requests

from dataclasses import dataclass, field

import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p
from itertools import combinations
from typing import List, Tuple
from enum import Enum
from scipy.stats import kendalltau, spearmanr

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, 'data')
output_file = join(data_dir, "nutri_food_data.csv")
parent_dir

'/Users/mohammadzainabbas/Masters/CS/Decision-Modelling/nutri-score'

In [9]:
!wget https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv -O ~/Masters/CS/Decision-Modelling/nutri-score/data/raw_products.csv

--2022-11-14 11:05:54--  https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
Resolving static.openfoodfacts.org (static.openfoodfacts.org)... 64:ff9b::d524:fdce, 213.36.253.206
Connecting to static.openfoodfacts.org (static.openfoodfacts.org)|64:ff9b::d524:fdce|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7325177980 (6.8G) [text/csv]
Saving to: ‘/Users/mohammadzainabbas/Masters/CS/Decision-Modelling/nutri-score/data/raw_products.csv’

s/CS/Decision-Model   1%[                    ] 103.23M   404KB/s    eta 2h 52m ^C


In [11]:
res = requests.get("https://world.openfoodfacts.org/api/v0/product/737628064502.json")
res

<Response [200]>

In [15]:
res = res.json()

In [23]:
column_mapping = {
    "brand": "product.brand_owner",

}

In [24]:
def get_mapping(data, value):
    values = value.split(".")
    _data = None
    for val in values:
        _data = _data[val]
    return _data

In [25]:
get_mapping(res, column_mapping['brand'])

KeyError: 'brand_owner'

In [20]:
res["product"]["brand_owner"]

'Simply Asia Foods, Inc.'